In [1]:
%matplotlib inline
from __future__ import division

import pandas as pd
import utils

In [2]:
visitors, events, devices, url_categories = utils.load_data(event_categories=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
categorized_visits = pd.merge(visitors, events, on='dg_id')
categorized_visits['authorized'] = categorized_visits.identity_type.apply(lambda x: 1 if x=='AUT' else 0)

In [4]:
pharma = categorized_visits[categorized_visits.site_category=='Pharma']
pharma_activity = pharma.groupby('category_url').authorized.agg(['sum', 'count']).reset_index()
pharma_activity.columns = ['url', 'authorized_visits', 'total_visits']
pharma_activity = pd.merge(url_categories, pharma_activity, on='url').fillna('')
pharma_activity['pct_authorized'] = pharma_activity.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)

In [7]:
def authorized_visits_print(pharma_site, pharma_activity):
    f = pharma_activity[pharma_activity.url==pharma_site]
    print 'Pharma Site :: ' + pharma_site + ' :: ' + str(round(f.pct_authorized*100,2)) + '% authorized'
    disease_category = f.disease_category.item()
    fd = pharma_activity[pharma_activity.disease_category==disease_category]
    if len(fd) >= 3:
        authorized = sum(fd.authorized_visits)
        total = sum(fd.total_visits)
        print 'Disease Category :: ' + disease_category + ' :: ' + str(round(authorized/total*100, 2)) + '% authorized'
        diseases = [x.lower().strip() for x in f.disease.item().split(',') if x.strip() != '']
        for d in diseases:
            authorized = 0
            total = 0
            count = 0
            for row in fd.iterrows():
                r = row[1]
                if d in r.disease.lower():
                    authorized += r.authorized_visits
                    total += r.total_visits
                    count += 1
            if count >= 3:
                print 'Disease :: ' + d + ' :: ' + str(round(authorized/total*100, 2)) + '% authorized'
                

In [62]:
pharma_activity['pharma_id'] = range(len(pharma_activity))

In [69]:
def authorized_visits_dict(pharma_site, pharma_activity):
    compare = []
    f = pharma_activity[pharma_activity.url==pharma_site]
    d = {}
    d['pharma_id'] = f.pharma_id.item()
    d['level'] = 'main pharma site'
    d['name'] = pharma_site
    d['pct_authorized'] = round(f.pct_authorized*100,2)
    compare.append(d)
    disease_category = f.disease_category.item()
    fd = pharma_activity[pharma_activity.disease_category==disease_category]
    if len(fd) >= 3:
        d = {}
        authorized = sum(fd.authorized_visits)
        total = sum(fd.total_visits)
        d['pharma_id'] = f.pharma_id.item()
        d['level'] = 'disease category'
        d['name'] = disease_category 
        d['pct_authorized'] = round(authorized/total*100, 2)
        d['number_of_sites'] = len(fd)  
        compare.append(d)
        diseases = [x.lower().strip() for x in f.disease.item().split(',') if x.strip() != '']
        for t in diseases:
            authorized = 0
            total = 0
            count = 0
            for row in pharma_activity.iterrows():
                r = row[1]
                if t in r.disease.lower():
                    authorized += r.authorized_visits
                    total += r.total_visits
                    count += 1
            if count >= 3:
                d = {}
                d['pharma_id'] = f.pharma_id.item()
                d['level'] = 'disease'
                d['name'] = t
                d['pct_authorized'] = round(authorized/total*100, 2)
                d['number_of_sites'] = count
                compare.append(d)
    return compare

In [70]:
authorized_visits_dict(url, pharma_activity)

[{'level': 'main pharma site',
  'name': 'www.zostavax.com',
  'pct_authorized': 68.76,
  'pharma_id': 114},
 {'level': 'disease category',
  'name': 'Infectious Disease',
  'number_of_sites': 10,
  'pct_authorized': 76.01,
  'pharma_id': 114}]

In [71]:
all_urls = []
for url in pharma_activity.url:
    all_urls += authorized_visits_dict(url, pharma_activity)

In [72]:
df = pd.DataFrame(all_urls)

In [74]:
df.to_csv('authorized_visits_by_disease.csv')

In [5]:
unk = pd.read_csv('../data/unqualified_traffic_summary.csv')

In [6]:
unk.head()

,date,domain,sessions,unique_urls,pageviews,unqualified
0,﻿2017-02-01,NaN,5,5,5,Y
1,2017-02-01,0-press.endocrine.org.elibrary.qatar-weill.cor...,1,1,2,Y
2,2017-02-01,0-press.endocrine.org.fama.us.es,1,2,2,N
3,2017-02-01,0-press.endocrine.org.libus.csd.mu.edu,1,1,1,N
4,2017-02-01,10.60.2.43,1,9,9,N


In [9]:
unk.unqualified.value_counts()

N    49787
Y     3733
Name: unqualified, dtype: int64

In [10]:
unk.sample(10)

,date,domain,sessions,unique_urls,pageviews,unqualified
7282,2017-02-22,press.endocrine.org.pbidi.unam.mx,2,2,2,N
29947,2017-05-04,www.clinicaloncology.com,140,74,308,N
19184,2017-03-27,theoncologist.alphamedpress.org.proxy.lib.umic...,1,1,2,N
30221,2017-05-05,www.knowyourprostateplan.com,130,120,390,N
31311,2017-05-10,ophth.broadcastmed.com,8,7,13,N
41079,2017-06-14,flip.it,2,2,2,N
20829,2017-04-01,theoncologist.alphamedpress.org,4632,2278,6760,N
37840,2017-06-02,www.healthjobsnationwide.com,2342,7216,11201,N
40415,2017-06-12,endonews.matrixdev.net,1,1,1,Y
47699,2017-07-07,endocrinenews.endocrine.org,667,148,1066,N


In [37]:
unk['stripped_url'] = unk.domain.apply(lambda x: utils.clean_url(x, strip=True))

In [44]:
unk_overall = unk.groupby('stripped_url').sum().reset_index()
unk_overall.columns = ['stripped_url', 'unk_sesions', 'unk_unique_urls', 'unk_pageviews']

In [45]:
unk_overall.head()

,stripped_url,unk_sesions,unk_unique_urls,unk_pageviews
0,0,3,5,19
1,0-endocrine,4,6,15
2,0-journals,1,1,1
3,0-press,252,259,371
4,0-theoncologist,2,7,10


In [42]:
url_categories = pd.merge(url_categories, unk_overall, on='stripped_url', how='left')

In [36]:
unk[unk.domain.str.contains('hcp')]

,date,domain,sessions,unique_urls,pageviews,unqualified,stripped_url
68,2017-02-01,hcp.cabometyx.com,110,37,186,N,hcp/hcp
350,2017-02-01,www.myrbetriqhcp.com,68,7,81,N,myrbetriqhcp
379,2017-02-01,www.vesicarehcp.com,25,5,28,N,vesicarehcp
388,2017-02-01,www.xtandihcp.com,468,343,875,N,xtandihcp
393,2017-02-01,xtandihcp.com,23,34,48,N,xtandihcp
455,2017-02-02,hcp.cabometyx.com,238,175,338,N,hcp/hcp
739,2017-02-02,www.myrbetriqhcp.com,72,9,95,N,myrbetriqhcp
772,2017-02-02,www.vesicarehcp.com,26,6,49,N,vesicarehcp
779,2017-02-02,www.xtandihcp.com,408,274,871,N,xtandihcp
782,2017-02-02,xtandihcp.com,42,40,104,N,xtandihcp
